# PHD - DISC24 - Exp 4 - KL for benchmark circuits - native gates - v1 - 20240406

//======================================================================================================
// 

EPUSP - Quantum circuits dimensions - v.04.2024 
Author: Waldemir Cambiucci 
Data: April 4th, 2024

File: PHD - DISC24 - Exp 4 - KL for benchmark circuits - native gates - v1 - 20240406

Benchmark circuits:
- Amplitude Estimation (AE)
- Grover's Algorithm (No Ancilla)
- Quantum Approximation Optimization Algorithm (QAOA)
- Quantum Fourier Transformation (QFT)
- Quantum Phase Estimation (QPE) – Exact
- Variational Quantum Eigensolver (VQE)

Main Folder: disc_exp4_benchmark_native_circuits

AE : qubits[4..120] qreg(eval, q)
- opt0
- opt1
- opt2
- opt3
template file: ae_nativegates_ibm_qiskit_opt0_4.qasm

GROVER : qubits[4..10] qreg(q, flag) 
- opt0
- opt1
- opt2
- opt3
template file: grover-noancilla_nativegates_ibm_qiskit_opt3_4.qasm

QAOA : qubits[4..16] qreg(q)
- opt0
- opt1
- opt2
- opt3
template file: qaoa_nativegates_ibm_qiskit_opt0_4.qasm

QFT :  qubits[4..120] qreg(q)
- opt0
- opt1
- opt2
- opt3
template file: qft_nativegates_ibm_qiskit_opt0_4.qasm

QPE :  qubits[4..120] qreg(q, psi)
- opt0
- opt1
- opt2
- opt3
template file: qpeexact_nativegates_ibm_qiskit_opt0_4.qasm

VQE :  qubits[4..16] qreg(q)
- opt0
- opt1
- opt2
- opt3
template file: vqe_nativegates_ibm_qiskit_opt0_4.qasm

https://www.calculatorsoup.com/calculators/discretemathematics/combinations.php


//
//======================================================================================================

In [2]:
import re
import networkx as nx
from qiskit import QuantumCircuit
from networkx.algorithms import community

In [3]:
def extract_qubits(input_str):
    # Regular expression to match qubit numbers
    pattern = r'q\[(\d+)\]'
    # Find all matches of the pattern in the input string
    matches = re.findall(pattern, input_str)
    # Extract qubit numbers from matches
    qubit_numbers = [int(match) for match in matches]
    # Return the qubit numbers as separate variables
    if len(qubit_numbers) >= 2:
        return tuple(qubit_numbers)
    elif len(qubit_numbers) == 1:
        return qubit_numbers[0], None
    else:
        return None, None

In [4]:
def get_num_qubits_from_qasm(qasm_str):  
    for line in qasm_str.splitlines():
        if line.startswith('qreg'):        
            pos = line.find('[') + 1
            end_pos = line.find(']')
            qubits_str = line[pos:end_pos]    
    return int(qubits_str)

In [5]:
import networkx as nx
def convert_qasm_in_hypergraph_kl(qasm_str):
    g = nx.MultiGraph()     
    for line in qasm_str.splitlines():      
        if line.startswith('cx'):        
            qubit1, qubit2 = extract_qubits(line)      
            g.add_edge(qubit1,qubit2)      
    return g           

In [6]:
def count_ebits(g , b):      
    nebits = 0
    for edge in g.edges:
        n1 = edge[0]
        n2 = edge[1]        
        if (n1 in b[0]) and (n2 in b[1]):   
            nebits = nebits + 1
        elif (n1 in b[1]) and (n2 in b[0]):   
            nebits = nebits + 1
    return nebits

In [7]:
def get_coupling_base_from_circuit(circuit):
    n = circuit.num_qubits
    Cb = (n * (n-1))/2  
    return Cb

In [8]:
def get_coupling_ratio_from_circuit(circuit):
    num_bin_gates = count_binary_operations(circuit)
    Cb = get_coupling_base_from_circuit(circuit)
    Cr = num_bin_gates / Cb
    return Cr

In [9]:
def count_binary_operations(quantum_circuit):
    # Count the operations using count_ops() function
    ops_dict = quantum_circuit.count_ops()    
    # Sum the counts of binary operations
    total_binary_ops = sum(count for op, count in ops_dict.items() if op in ['cx', 'cz', 'swap', 'cp'])    
    return total_binary_ops

In [10]:
def create_full_circuit(n):
    bc = QuantumCircuit(n)    
    total_unique_gates = (n*(n-1))/2     
    for q0 in range(0,n):
        bc.barrier()
        for q1 in range(q0+1,n):
            bc.cz(q0,q1)    
    return bc

In [11]:
def split_circuit_and_count_binary_gates(circuit):
    # Get the number of qubits in the circuit
    num_qubits = circuit.num_qubits
    
    # Calculate the midpoint to split the circuit
    midpoint = num_qubits // 2
    
    # Divide the qubits into two partitions
    partition_1 = list(range(midpoint))    
    partition_2 = list(range(midpoint, num_qubits))
    
    # Initialize count for binary gates involving qubits from both partitions
    count_binary_gates = 0
    
    # Iterate through the gates in the circuit
    for instr, qargs, cargs in circuit.data:
        # Check if the gate is a binary gate and involves qubits from both partitions
        if len(qargs) == 2 and (qargs[0].index in partition_1 and qargs[1].index in partition_2) or \
           (qargs[0].index in partition_2 and qargs[1].index in partition_1):
            count_binary_gates += 1
    
    return count_binary_gates

In [12]:
from qiskit import QuantumCircuit
def split_circuit_and_count_binary_gates(circuit):
    # Get the number of qubits in the circuit
    num_qubits = circuit.num_qubits
    
    # Calculate the midpoint to split the circuit
    midpoint = num_qubits // 2
    
    # Divide the qubits into two partitions
    partition_1 = set(range(midpoint))    
    partition_2 = set(range(midpoint, num_qubits))
    
    # Initialize count for binary gates involving qubits from both partitions
    count_binary_gates = 0
    
    # Iterate through each gate in the circuit
    for gate in circuit:
        # Extract the qubits associated with the gate
        qubits_involved = {qubit.index for qubit in gate[1]}
        
        # Check if the gate is a binary gate and involves qubits from both partitions
        if len(qubits_involved) == 2 and (qubits_involved & partition_1) and (qubits_involved & partition_2):
            count_binary_gates += 1
    
    return count_binary_gates

# DISC24 - Exp 4 - KL for benchmark circuits with native gates
# 
# Starting with QAOA (4..16 qubits), QFT (4..120qubits), VQE (4..16qubits)

In [13]:
from datetime import datetime
def generate_timestamp():
    """
    Generates a timestamp string representing the current date and time.    
    Returns:
        str: Timestamp string in the format 'YYYY-MM-DD_HH-MM-SS'.
    """
    now = datetime.now()
    timestamp = now.strftime('%Y-%m-%d_%H-%M-%S')
    return timestamp

In [23]:
def disc_exp4_benchmar_circuits_native_gates(mainfolder, subfolder, preStr, opt, minqubit, maxqubit):
    
    timestamp = generate_timestamp()
    print("Timestamp:", timestamp)
    
    midStr = "nativegates_ibm_qiskit_opt"
    sufStr=".qasm"

    print('mainfolder: ', mainfolder) 
    print('subfolder: ', subfolder)
    print('preStr: ', preStr)
    print('opt: ', opt)
    print('minqubit: ', minqubit)
    print('maxqubit: ',maxqubit)

    # Folder of circuit files for benchmark.
    circuit_folder_opt = mainfolder + "\\" + subfolder + "\\opt" + str(opt)

    print('Running experiment 4 - benchmark circuits with native gates.')
    print('Benchmark circuit: ', subfolder)
    print('Optimization - opt: ' + str(opt))
    print('----------------------------------------------------------------------------------------------------------')
    print('...')

    file1 = open(circuit_folder_opt + "\\" + "exp_4_benchmark_native_" + subfolder + "_opt" + str(opt) + '-' + timestamp + '.log', 'w')
    
    # Write the contents of the list to a file named 'coupling.log'
    file1.writelines('benchmark native circuit' + '\t' + 'width (n)' + '\t' + 'depth (d)' + '\t' + 'size (s)' + '\t' + 'coupling (Cb)' + '\t' + 'mid.cut' + '\t' + 'min.cut KL'+ '\t' + 'bipartite segments' + '\n')


    #=====================================================================
    #
    for circuit_file_index in range(minqubit,maxqubit):

        benchmark_filename =  preStr + midStr + str(opt) + "_" + str(circuit_file_index) + sufStr        
        print('Benchmark native circuit :', benchmark_filename)

        # loading qasm instructions for the circuit.
        circuit = QuantumCircuit.from_qasm_file( circuit_folder_opt + "\\" + benchmark_filename )            

        d = circuit.depth()
        s = circuit.size()       
        n = circuit.num_qubits
        
        mid_cuts = split_circuit_and_count_binary_gates(circuit)
        
        Cb = get_coupling_base_from_circuit(circuit)    
        Rc = get_coupling_ratio_from_circuit(circuit)        
        
        qasm_str = circuit.qasm()
        qasm_hypergraph = convert_qasm_in_hypergraph_kl(qasm_str)

        blocks = community.kernighan_lin.kernighan_lin_bisection(qasm_hypergraph, partition=None, max_iter=50)
        num_ebits = count_ebits(qasm_hypergraph,blocks)    

        file1.writelines( benchmark_filename + '\t' + str(n) + '\t' + str(d) + '\t' + str(s) + '\t' + str(Cb) + '\t' + str(mid_cuts) + '\t' + str(num_ebits) + '\t' + str(blocks) + '\n' )

    #
    #=====================================================================

    print('...')
    print('File closed.')

    file1.close()
    

In [ ]:
for optidx in range(0,4):
    disc_exp4_benchmar_circuits_native_gates("disc_exp4_benchmark_native_circuits", "QFT", "qft_", optidx, 4, 16)

In [29]:
for optidx in range(0,4):
    disc_exp4_benchmar_circuits_native_gates("disc_exp4_benchmark_native_circuits", "QAOA", "qaoa_", optidx, 4, 16)

Timestamp: 2024-04-06_18-45-22
mainfolder:  disc_exp4_benchmark_native_circuits
subfolder:  QAOA
preStr:  qaoa_
opt:  0
minqubit:  4
maxqubit:  16
Running experiment 4 - benchmark circuits with native gates.
Benchmark circuit:  QAOA
Optimization - opt: 0
----------------------------------------------------------------------------------------------------------
...
Benchmark native circuit : qaoa_nativegates_ibm_qiskit_opt0_4.qasm
Benchmark native circuit : qaoa_nativegates_ibm_qiskit_opt0_5.qasm
Benchmark native circuit : qaoa_nativegates_ibm_qiskit_opt0_6.qasm
Benchmark native circuit : qaoa_nativegates_ibm_qiskit_opt0_7.qasm
Benchmark native circuit : qaoa_nativegates_ibm_qiskit_opt0_8.qasm
Benchmark native circuit : qaoa_nativegates_ibm_qiskit_opt0_9.qasm
Benchmark native circuit : qaoa_nativegates_ibm_qiskit_opt0_10.qasm
Benchmark native circuit : qaoa_nativegates_ibm_qiskit_opt0_11.qasm
Benchmark native circuit : qaoa_nativegates_ibm_qiskit_opt0_12.qasm
Benchmark native circuit : q

C:\Users\wcamb\AppData\Local\Temp\ipykernel_412\1118521752.py:19: DeprecationWarning: The property ``qiskit.circuit.bit.Bit.index`` is deprecated as of qiskit-terra 0.17. It will be removed no earlier than 3 months after the release date. Instead, use :meth:`~qiskit.circuit.quantumcircuit.QuantumCircuit.find_bit` to find all the containing registers within a circuit and the index of the bit within the circuit.
  qubits_involved = {qubit.index for qubit in gate[1]}


Benchmark native circuit : qaoa_nativegates_ibm_qiskit_opt0_14.qasm
Benchmark native circuit : qaoa_nativegates_ibm_qiskit_opt0_15.qasm
...
File closed.
Timestamp: 2024-04-06_18-45-22
mainfolder:  disc_exp4_benchmark_native_circuits
subfolder:  QAOA
preStr:  qaoa_
opt:  1
minqubit:  4
maxqubit:  16
Running experiment 4 - benchmark circuits with native gates.
Benchmark circuit:  QAOA
Optimization - opt: 1
----------------------------------------------------------------------------------------------------------
...
Benchmark native circuit : qaoa_nativegates_ibm_qiskit_opt1_4.qasm
Benchmark native circuit : qaoa_nativegates_ibm_qiskit_opt1_5.qasm
Benchmark native circuit : qaoa_nativegates_ibm_qiskit_opt1_6.qasm
Benchmark native circuit : qaoa_nativegates_ibm_qiskit_opt1_7.qasm
Benchmark native circuit : qaoa_nativegates_ibm_qiskit_opt1_8.qasm
Benchmark native circuit : qaoa_nativegates_ibm_qiskit_opt1_9.qasm
Benchmark native circuit : qaoa_nativegates_ibm_qiskit_opt1_10.qasm
Benchmark n

In [28]:
for optidx in range(0,4):
    disc_exp4_benchmar_circuits_native_gates("disc_exp4_benchmark_native_circuits", "VQE", "vqe_", optidx, 4, 16)

Timestamp: 2024-04-06_18-44-52
mainfolder:  disc_exp4_benchmark_native_circuits
subfolder:  VQE
preStr:  vqe_
opt:  0
minqubit:  4
maxqubit:  16
Running experiment 4 - benchmark circuits with native gates.
Benchmark circuit:  VQE
Optimization - opt: 0
----------------------------------------------------------------------------------------------------------
...
Benchmark native circuit : vqe_nativegates_ibm_qiskit_opt0_4.qasm
Benchmark native circuit : vqe_nativegates_ibm_qiskit_opt0_5.qasm
Benchmark native circuit : vqe_nativegates_ibm_qiskit_opt0_6.qasm
Benchmark native circuit : vqe_nativegates_ibm_qiskit_opt0_7.qasm
Benchmark native circuit : vqe_nativegates_ibm_qiskit_opt0_8.qasm
Benchmark native circuit : vqe_nativegates_ibm_qiskit_opt0_9.qasm
Benchmark native circuit : vqe_nativegates_ibm_qiskit_opt0_10.qasm
Benchmark native circuit : vqe_nativegates_ibm_qiskit_opt0_11.qasm
Benchmark native circuit : vqe_nativegates_ibm_qiskit_opt0_12.qasm
Benchmark native circuit : vqe_nativegat

C:\Users\wcamb\AppData\Local\Temp\ipykernel_412\1118521752.py:19: DeprecationWarning: The property ``qiskit.circuit.bit.Bit.index`` is deprecated as of qiskit-terra 0.17. It will be removed no earlier than 3 months after the release date. Instead, use :meth:`~qiskit.circuit.quantumcircuit.QuantumCircuit.find_bit` to find all the containing registers within a circuit and the index of the bit within the circuit.
  qubits_involved = {qubit.index for qubit in gate[1]}


Benchmark native circuit : vqe_nativegates_ibm_qiskit_opt0_14.qasm
Benchmark native circuit : vqe_nativegates_ibm_qiskit_opt0_15.qasm
...
File closed.
Timestamp: 2024-04-06_18-44-53
mainfolder:  disc_exp4_benchmark_native_circuits
subfolder:  VQE
preStr:  vqe_
opt:  1
minqubit:  4
maxqubit:  16
Running experiment 4 - benchmark circuits with native gates.
Benchmark circuit:  VQE
Optimization - opt: 1
----------------------------------------------------------------------------------------------------------
...
Benchmark native circuit : vqe_nativegates_ibm_qiskit_opt1_4.qasm
Benchmark native circuit : vqe_nativegates_ibm_qiskit_opt1_5.qasm
Benchmark native circuit : vqe_nativegates_ibm_qiskit_opt1_6.qasm
Benchmark native circuit : vqe_nativegates_ibm_qiskit_opt1_7.qasm
Benchmark native circuit : vqe_nativegates_ibm_qiskit_opt1_8.qasm
Benchmark native circuit : vqe_nativegates_ibm_qiskit_opt1_9.qasm
Benchmark native circuit : vqe_nativegates_ibm_qiskit_opt1_10.qasm
Benchmark native circui